# Merge Evaluation Dataset

I made a mistake when creating the datasets for finetuning.

I made the instructions, prompts, responses part of their own datasets.

I should have instead made them new columns as part of a copy of the original.

So, I need to merge the evaluation results to the original dataset. Apologies.

I will fix this later.

In [41]:
import re
import ast
from datasets import load_dataset

In [75]:
original_class_dataset = load_dataset(
    "Distaste1194/csharp_test_classes_original",
    data_dir="data",
    split="train"
)

class_eval_dataset = load_dataset(
    "Distaste1194/llama-3.1-a100-csharp_test_classes_eval",
    data_dir="data",
    split="train"
)

In [76]:
original_method_dataset = load_dataset(
    "Distaste1194/csharp_test_methods_original",
    data_dir="data",
    split="train"
)

method_eval_dataset = load_dataset(
    "Distaste1194/llama-3.1-a100-csharp_test_methods_eval",
    data_dir="data",
    split="train"
)

In [46]:
original_class_df = original_class_dataset.to_pandas()
class_eval_df = class_eval_dataset.to_pandas()

original_method_df = original_method_dataset.to_pandas()
method_eval_df = method_eval_dataset.to_pandas()

## Classes

In [77]:
# Merging based on the 'original_index' column in df1
merged_class_df = class_eval_df.merge(original_class_df, how='inner', left_on='Original Index', right_index=True)


In [78]:
merged_class_df = merged_class_df.drop(columns=['Instructions', 'Original Index', 'Prompt', 'Expected_Response', '__index_level_0__', 'Repo_x', 'ClassBody'])

In [79]:
# Define a function to format and return the result
def extract_gen_test_class(row):
    class_body = row['Generated_Response'][0].split("<|im_end|>\n<|im_start|>assistant:")[-1].split("<|im_end|>")[0]
    formatted = class_body.replace("```", "")
    result = re.sub(r'^Here is the .*\n?', '', formatted, flags=re.MULTILINE)
    return result

# Using apply with the custom function to create a new column
merged_class_df['ClassBody'] = merged_class_df.apply(extract_gen_test_class, axis=1)

## Methods

In [80]:
def create_method_question(row):
    method_invocations = ast.literal_eval(row['MethodInvocations'])
    # Joining method and description, discarding if description is missing
    source_code = '\n'.join(
        f"Method: {method}\nDefinition: {description}"
        for method, description in method_invocations
        if description  # This checks if description is not None and not an empty string
    )
    
    # Check if source_code is empty or consists of blank lines
    if not source_code.strip():  # strip() removes whitespace
        return None
    
    class_fields = '\n'.join(ast.literal_eval(row['ClassFields']))
    using_statements = '\n'.join(ast.literal_eval(row['UsingStatements']))
    question = f"""

Here is some contextual information:
Source Code and definitions (if any):
{source_code}

Test Namespace: {row['Namespace']}

Test Class Declaration: {row['ClassDeclaration']}

Test Class Fields: 
{class_fields}

Test Using Statements: 
{using_statements}

Test Framework: {row['TestFramework']}

Language Framework: {row['LanguageFramework']}

Please delimit the code with ```
    """
    return question

In [81]:
original_method_df['question'] = original_method_df.apply(create_method_question, axis=1)


In [90]:
# Removing rows where 'question' is None
original_method_df = original_method_df.dropna(subset=['question'])
original_method_df = original_method_df.reset_index(drop=True)
original_method_df = original_method_df.drop(columns=['question'])

In [91]:
# Merging based on the 'original_index' column in df1
merged_method_df = method_eval_df.merge(original_method_df, how='inner', left_on='Original Index', right_index=True)


In [92]:
merged_method_df = merged_method_df.drop(columns=['Instructions', 'Original Index', 'Prompt', '__index_level_0__', 'Repo_x', 'MethodBody', 'Expected_Response'])

In [93]:
# Define a function to format and return the result
def extract_gen_test_method(row):
    method_body = row['Generated_Response'][0].split("<|im_start|>", 1)[-1]
    formatted = method_body.split("<|im_start|>", 1)[-1].split("<|im_end|>")[0]
    formatted = formatted.replace("```", "")
    return formatted

# Using apply with the custom function to create a new column
merged_method_df['MethodBody'] = merged_method_df.apply(extract_gen_test_method, axis=1)

In [94]:
merged_class_df = merged_class_df.drop(columns=['Generated_Response'])
merged_method_df = merged_method_df.drop(columns=['Generated_Response'])

KeyError: "['Generated_Response'] not found in axis"

In [95]:
merged_class_df.rename(columns={'Repo_y': 'Repo'}, inplace=True)
merged_method_df.rename(columns={'Repo_y': 'Repo'}, inplace=True)

## Save

In [89]:

merged_class_df.to_csv("merged_class_df.csv", index=False)
merged_method_df.to_csv("merged_method_df.csv", index=False)